# *F* test
---

In [1]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Stats imports
from statsmodels.stats.power import FTestPower

# MDE Derivation
---
The normalized effect size commonly used for the *F* test (and implemented in `statsmodels`) is Cohen's *f*. This metric is defined as:

$$f=\sqrt{\frac{\sigma_m^2}{\sigma^2}}=\sqrt{\frac{\Sigma_it_i(\mu_i-\bar{\mu})^2}{\sigma^2}}=\sqrt{\frac{\Sigma_it_i(\mu_i-\Sigma_kt_k(\mu+e_k))^2}{\sigma^2}},$$

where $e_i, t_i$ are the absolute effect size and test split for experimental group $i$.

We find that the minimal detectable effect (MDE) for test cell $j$ can be found by minimizing $e_j$. To do this, we set the equation for Cohen's *f* to zero and (through a bit of algebra) solve for the following second-order polynomial:

$$0 = e_j^2[t_j-t_j^2] + e_j[-2t_jX]+[X^*-X^2-f^2\sigma^2],$$

where $\Sigma^*$ is the summation of all experimental groups excluding fixed group $j$, which is being minimized; $X=\Sigma_k^*t_ke_k$; and $X^*=\Sigma^*_it_ie_i^2$.

Let's first solve for Cohen's *f*:

In [2]:
def cohens_f(test_splits, mus, sigma):
    test_splits = np.array(test_splits)
    assert round(test_splits.sum(), 6) == 1., "Test splits must sum to 1"
    mus = np.array(mus)
    pop_mu = (mus * test_splits).sum()
    sigma2_m = (test_splits * np.square(mus - pop_mu)).sum()
    f = np.sqrt(sigma2_m) / sigma
    return f

mu = 8.
sigma = 1.34
absolute_effects = [4., 3., 0.]
test_splits = [0.4, 0.4, 0.2]

mus = [mu + ae for ae in absolute_effects]
effect_size = cohens_f(test_splits, mus, sigma)
print("Normalized effect size (Cohen's f) is:", effect_size)

Normalized effect size (Cohen's f) is: 1.0967864519924677


Next, let's check that the math in our second-order polynomial holds:

In [10]:
def check_math_1(test_splits, absolute_effects, mu, sigma):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    mus = np.array([mu + ae for ae in absolute_effects])
    
    f = cohens_f(test_splits, mus, sigma)
    
    index = 0 # Fix any test group
    t_j = test_splits[index]
    e_j = absolute_effects[index]
    
    X = (test_splits * absolute_effects).sum() - (t_j * e_j)
    X_star = (test_splits * absolute_effects * absolute_effects).sum() - (t_j * e_j * e_j)
    
    output = e_j**2 * (t_j - t_j**2) + e_j * (-2 * t_j * X) + (X_star - X**2 - f**2 * sigma**2)
    print("Algebraic output: %.8f" % output)

check_math_1(test_splits, absolute_effects, mu, sigma)

Algebraic output: 0.00000000


Now let's solve for the roots of $e_j$ to minimize the effect size:

In [5]:
def solve_absolute_mde_roots(sample_size, test_splits, absolute_effects, mu, sigma, alpha, beta):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    
    df_num = len(test_splits) - 1
    df_denom = sample_size - len(test_splits)
    
    f = FTestPower().solve_power(
        effect_size=None
        ,df_num=df_denom
        ,df_denom=df_num
        ,alpha=alpha
        ,power=(1-beta)
        ,ncc=1
    )
    
    index = 0 # Fix any test group
    t_j = test_splits[index]
    e_j = absolute_effects[index]
    
    X = (test_splits * absolute_effects).sum() - (t_j * e_j)
    X_star = (test_splits * absolute_effects * absolute_effects).sum() - (t_j * e_j * e_j)
    
    a = t_j - t_j**2
    b = -2 * t_j * X
    c = X_star - X**2 - f**2 * sigma**2
    
    roots = np.roots([a, b, c])
    return roots

# Experimental parameters
sample_size = 132
test_splits = [0.8333, 0.1667]
absolute_effects = [-2.0, 0.0]
mu = 16.
sigma = 4.5
alpha = 0.10
beta = 0.20

absolute_mde_roots = solve_absolute_mde_roots(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
    ,beta=beta
)
print("Absolute MDE roots are:", absolute_mde_roots)

Absolute MDE roots are: [ 2.62662565 -2.62662565]


In [6]:
def solve_power(sample_size, test_splits, absolute_effects, mu, sigma, alpha):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    mus = np.array([mu + ae for ae in absolute_effects])
    
    df_num = len(test_splits) - 1
    df_denom = sample_size - len(test_splits)
    
    f = cohens_f(test_splits, mus, sigma)
    power = FTestPower().solve_power(
        effect_size=f
        ,df_num=df_denom
        ,df_denom=df_num
        ,alpha=alpha
        ,power=None
        ,ncc=1
    )
    return power

current_power = solve_power(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
)
print("Originally, the power of our system is:", current_power)

Originally, the power of our system is: 0.598281300868307


In [7]:
new_absolute_effects__positive_root = np.array([np.max(absolute_mde_roots), 0.])
new_absolute_effects__negative_root = np.array([np.min(absolute_mde_roots), 0.])

new_power__positive_root = solve_power(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=new_absolute_effects__positive_root
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
)
new_power__negative_root = solve_power(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=new_absolute_effects__negative_root
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
)

print("The desired power for the system is:", (1-beta))
print("The power of the system with the positive root is:", new_power__positive_root)
print("The power of the system with the negative root is:", new_power__negative_root)

The desired power for the system is: 0.8
The power of the system with the positive root is: 0.7999947178139011
The power of the system with the negative root is: 0.7999947178139005


In [8]:
def solve_absolute_mdes(sample_size, test_splits, absolute_effects, mu, sigma, alpha, beta):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    
    n_groups = len(test_splits)
    df_num = n_groups - 1
    df_denom = sample_size - n_groups
    
    f = FTestPower().solve_power(
        effect_size=None
        ,df_num=df_denom
        ,df_denom=df_num
        ,alpha=alpha
        ,power=(1-beta)
        ,ncc=1
    )
    
    mde_list = []
    for index in range(n_groups):
        t_j = test_splits[index]
        e_j = absolute_effects[index]

        X = (test_splits * absolute_effects).sum() - (t_j * e_j)
        X_star = (test_splits * absolute_effects * absolute_effects).sum() - (t_j * e_j * e_j)

        a = t_j - t_j**2
        b = -2 * t_j * X
        c = X_star - X**2 - f**2 * sigma**2

        roots = np.roots([a, b, c])
        
        if np.iscomplex(roots).sum() > 0:
            print("MDE calculation found complex roots; returning closest effect size to desired power.")
            min_absolute_effect = roots[0].real
        elif e_j >= 0: min_absolute_effect = np.max(roots)
        else: min_absolute_effect = np.min(roots)
        
        mde_list.append(min_absolute_effect)
    return mde_list

absolute_mdes = solve_absolute_mdes(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
    ,beta=beta
)
print("List of absolute MDEs is:", absolute_mdes)

List of absolute MDEs is: [-2.6266256463594373, 0.6266256463594378]


# Linear MDE Solution
---
We may not want to solve for only one specific test cell. Instead, we may want to solve for the MDE across *all* test groups. The user typically makes an assumption for the effect sizes a priori based on evidence or instinct. We will leverage these initial assumptions, then **scale them linearly** to achieve the desired power.

Reorganizing Cohen's *f* equation,

$$f^2\sigma^2 = \Sigma_it_i(\mu_i-\bar{\mu})^2=\Sigma_it_i(e_i-\Sigma_kt_ke_k)^2$$

For each effect size $e$, we substitute a linearly scaled ($a$) effect size $ae$:

$$f^2\sigma^2 = a^2\Sigma_it_i(e_i-\Sigma_kt_ke_k)^2$$

In [9]:
def solve_absolute_mdes_linear(sample_size, test_splits, absolute_effects, mu, sigma, alpha, beta):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    
    n_groups = len(test_splits)
    df_num = n_groups - 1
    df_denom = sample_size - n_groups
    
    f = FTestPower().solve_power(
        effect_size=None
        ,df_num=df_denom
        ,df_denom=df_num
        ,alpha=alpha
        ,power=(1-beta)
        ,ncc=1
    )
    
    Y = (test_splits * absolute_effects).sum()
    num1 = np.square(absolute_effects - Y)
    num = (test_splits * num1).sum()
    
    a = f * sigma / np.sqrt(num)
    
    if absolute_effects.sum() < 0: a*= -1
    
    return a * absolute_effects

absolute_mdes_linear = solve_absolute_mdes_linear(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
    ,beta=beta
)
print("Absolute MDEs, linearly scaled:", absolute_mdes_linear)

Absolute MDEs, linearly scaled: [ 2.62662565 -0.        ]


# Optimal Test Split Derivation
---
Using the MDE solution for test group $j$, we can reorder the terms such that:

$$0 = t^2_j[-e_j^2] + t_j[e_j^2-2e_jX]+[X^*-X^2-f^2\sigma^2]$$

We check the math below:

In [11]:
def check_math_2(test_splits, absolute_effects, mu, sigma):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    mus = np.array([mu + ae for ae in absolute_effects])
    
    f = cohens_f(test_splits, mus, sigma)
    
    index = 0 # Fix any test group
    t_j = test_splits[index]
    e_j = absolute_effects[index]
    
    X = (test_splits * absolute_effects).sum() - (t_j * e_j)
    X_star = (test_splits * absolute_effects * absolute_effects).sum() - (t_j * e_j * e_j)
    
    a = -1 * e_j**2
    b = e_j**2 - 2 * e_j * X
    c = X_star - X**2 - f**2 * sigma**2
    
    output = t_j**2 * a + t_j * b + c
    print("Algebraic output: %.8f" % output)

check_math_2(test_splits, absolute_effects, mu, sigma)

Algebraic output: 0.00000000


In [12]:
def solve_test_splits(sample_size, test_splits, absolute_effects, mu, sigma, alpha, beta):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    
    n_groups = len(test_splits)
    df_num = n_groups - 1
    df_denom = sample_size - n_groups
    
    f = FTestPower().solve_power(
        effect_size=None
        ,df_num=df_denom
        ,df_denom=df_num
        ,alpha=alpha
        ,power=(1-beta)
        ,ncc=1
    )
    
    split_list = []
    for index in range(n_groups):
        t_j = test_splits[index]
        e_j = absolute_effects[index]
        
        if e_j == 0:
            print("Appending -1 for group with no experimental lift (group #"+str(index)+")")
            split_list.append(-1)
        else:
            X = (test_splits * absolute_effects).sum() - (t_j * e_j)
            X_star = (test_splits * absolute_effects * absolute_effects).sum() - (t_j * e_j * e_j)

            a = -1 * e_j**2
            b = e_j**2 - 2 * e_j * X
            c = X_star - X**2 - f**2 * sigma**2

            roots = np.roots([a, b, c])

            if np.iscomplex(roots).sum() > 0:
                print("MDE calculation found complex roots; returning closest effect size to desired power.")
                split = roots[0].real
            else: split = np.max(roots)

            split_list.append(split)
    return split_list

# Experimental parameters
sample_size = 132
test_splits = [0.8333, 0.1667]
absolute_effects = [-2.0, 0.0]
mu = 16.
sigma = 4.5
alpha = 0.10
beta = 0.30

opt_test_splits = solve_test_splits(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
    ,beta=beta
)
print("Optimized test splits are:", opt_test_splits)

Appending -1 for group with no experimental lift (group #1)
Optimized test splits are: [0.760121581677034, -1]


In [13]:
current_power = solve_power(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
)
print("Originally, the power of our system is:", current_power)

Originally, the power of our system is: 0.598281300868307


In [15]:
optimal_split = opt_test_splits[0]
new_test_splits = [optimal_split, 1-optimal_split]

new_power = solve_power(
    sample_size=sample_size
    ,test_splits=new_test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
)

print("The desired power of the system is:", (1-beta))
print("The power of the system with optimal split is:", new_power)

The desired power of the system is: 0.7
The power of the system with optimal split is: 0.7000002915296455


In [16]:
# Experimental parameters
sample_size = 400
test_splits = [0.4, 0.4, 0.2]
absolute_effects = [-2.0, -1.0, 0.0]
mu = 16.
sigma = 4.5
alpha = 0.10
beta = 0.20

opt_test_splits = solve_test_splits(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
    ,beta=beta
)
print("Optimized test splits are:", opt_test_splits)

Appending -1 for group with no experimental lift (group #2)
Optimized test splits are: [0.5276976330163264, 0.51079272833749, -1]


In [17]:
current_power = solve_power(
    sample_size=sample_size
    ,test_splits=test_splits
    ,absolute_effects=absolute_effects
    ,mu=mu
    ,sigma=sigma
    ,alpha=alpha
)
print("Originally, the power of our system is:", current_power)

Originally, the power of our system is: 0.9132807915248896


In [19]:
print("The desired power of the system is:", (1-beta))

control_group_index = opt_test_splits.index(-1)

for i, split in enumerate(opt_test_splits):
    if split != -1:
        new_test_splits = test_splits.copy()
        prev_test_split = test_splits[i]
        new_test_splits[i] = split
        new_test_splits[control_group_index] -= (split - prev_test_split)
        new_power = solve_power(
            sample_size=sample_size
            ,test_splits=new_test_splits
            ,absolute_effects=absolute_effects
            ,mu=mu
            ,sigma=sigma
            ,alpha=alpha
        )
        print("After updating test split for group #"+str(i)+" power is:", new_power)

The desired power of the system is: 0.8
After updating test split for group #0 power is: 0.7999991758656374
After updating test split for group #1 power is: 0.7999991758656372


# Minimize control split
---
A common request for a group running experimentation is to minimize the control split in oder to maximize the amount of potential value the treatment creates. The problem with minimizing the control group is that the way we inflate the other test groups ($>2$) is non-trivial. Let's assume that we would **increase the test groups linearly** as we minimize the control group.

From our system above,

$$f^2\sigma^2 = \Sigma_it_i(e_i-\Sigma_kt_ke_k)^2$$

Define $Y=\Sigma_kt_ke_k=\Sigma^*_kt_ke_k + e_jt_j=\Sigma_k^*t_ke_k$ where index $j$ corresponds with the control (hold-out) group ($e_j=0$). Thus,


$$f^2\sigma^2 = \Sigma_it_i(e_i-\Sigma_kt_ke_k)^2 = \Sigma_it_i(e_i-Y)^2$$

Partitioning out the control group $j$ yields:

$$\Sigma_it_i(e_i-Y)^2 = \Sigma_i^*t_i(e_i-Y)^2 + t_j(e_j-Y)^2=\Sigma_i^*t_ie_i^2-2Y\Sigma^*_it_ie_i+Y^2\Sigma_i^*t_i+t_jY^2$$

Since the test splits add to one ($\Sigma_kt_k=1$), then $\Sigma_i^*t_i = (1-t_j)$. Let's also define $Y^*=\Sigma^*_it_ie_i^2. Substitution yields

$$\Sigma_i^*t_ie_i^2-2Y^2+Y^2\Sigma^*_it_i+t_jY^2 = Y^*-2Y^2+Y^2(1-t_j)+t_jY^2=Y^*-Y^2$$

Thus, we find $f^2\sigma^2=Y^*-Y^2$. Because there are no control group splits $t_j$, we can solve for the factor $a$ that will linearly increase the splits:

$$a^2[Y^2]+a[-Y^*]+[f^2\sigma^2] = 0$$

In [24]:
def solve_test_split_control(test_splits, absolute_effects, mu, sigma):
    test_splits = np.array(test_splits)
    absolute_effects = np.array(absolute_effects)
    mus = np.array([mu + ae for ae in absolute_effects])
    
    f = cohens_f(test_splits, mus, sigma)
    
    Y = (test_splits * absolute_effects).sum()
    Y_star = (test_splits * absolute_effects * absolute_effects).sum()
    
    a = Y**2
    b = -1 * Y_star
    c = f**2 * sigma**2
    
    roots = np.roots([a, b, c])
    root = np.max(roots)
    
    control_index = np.argwhere(absolute_effects == 0)
    control_split = 1 - root + root * (test_splits[control_index].sum())
    
    if control_split <= 0:
        print("Forcing control group to at least 1%")
        root = 0.99 / (1- test_splits[control_index].sum())
        control_split = (1-root) + root * (test_splits[control_index].sum())
    
    opt_splits = root * test_splits
    opt_splits[control_index] = control_split / len(control_index)
    return opt_splits

test_split_control = solve_test_split_control(test_splits, absolute_effects, mu, sigma)
print("True test split:", test_splits[0])
print("Test split roots:", test_split_control)

True test split: 0.4
Test split roots: [0.4 0.4 0.2]
